<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/LPS%20Training%20-%20Exercise%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LPS hands-on training session: CoMet Toolkit: Uncertainties made easy**

#Exercise 2: Bigger Data

## Objectives

In this exercise we will cover:

* How to use [obsarray](https://obsarray.readthedocs.io/en/latest/) to store error-correlation information for multi-dimensional measurement datasets.
* Propagating uncertainties from these datasets through measurement functions using [punpy](https://punpy.readthedocs.io/en/latest/).

## Step 1 - Installing Requirements

As in Exercise 1, we start by install and importing the required CoMet Toolkit packages --- [obsarray](https://obsarray.readthedocs.io/en/latest/) and [punpy](https://punpy.readthedocs.io/en/latest/).

In [ ]:
# Install & import CoMet Toolkit packages
!pip install obsarray>=1.0.1
!pip install punpy>=1.0.4

import obsarray
import punpy
import xarray as xr

CoMet Toolkit's [**obsarray**](https://obsarray.readthedocs.io/en/latest/) package is an extension to the popular [xarray](https://docs.xarray.dev/en/stable/) package.

[`xarray.Dataset`](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#dataset)'s are objects in python that represent of the data model from the [netCDF file format](https://www.unidata.ucar.edu/software/netcdf/). **obsarray** allows you to assign uncertainties to variables in xarray Datasets, with their associated error-correlation.

This is achieved by using the CoMet Toolkit's [UNC Specification](https://comet-toolkit.github.io/unc_website/) metadata standard for uncertainty. So such objects are portable, and can be stored to and from disc.


## Step 2 - Creating a Measurement Dataset with obsarray

In this step of the exercise, we will explore how to set up measurement datasets with obsarray.